In [1]:
import amrlib
import pandas as pd
import penman
from   amrlib.graph_processing.amr_loading import load_amr_graph_sent

In [2]:
device      = 'cuda:0'  # or 'cpu'
batch_size  = 8
num_beams   = 8     # 1 ==> greedy
num_ret_seq = 1     # 1 ==> return best sentence only

In [3]:
def incoming_edge_count(variable, pgraph):
    incomings = [t for t in pgraph.edges() if t.target==variable]
    return len(incomings)

In [28]:
stog = "/cluster/home/abkumar/.local/lib/python3.10/site-packages/amrlib/model_stog/model_parse_xfm_bart_large-v0_1_0"
gtos = "/cluster/home/abkumar/.local/lib/python3.10/site-packages/amrlib/gtos/model_generate_t5wtense-v0_1_0"
stog = amrlib.load_stog_model(model_dir=stog)
gtos = amrlib.load_gtos_model(gtos, batch_size=batch_size, num_beams=num_beams, num_ret_seq=num_ret_seq)
graphs = stog.parse_sents(['The system can parse sentences to AMR graphs or generate text from existing graphs.', 'This is a second running sentence.'])

In [29]:
graphs[1], pgraph

('# ::snt This is a second running sentence.\n(s / sentence\n      :ARG1-of (r / run-04\n            :ord (o / ordinal-entity\n                  :value 2))\n      :domain (t / this))',
 <Graph object (top=p) at 47710788569024>)

In [30]:
pgraph = penman.decode(graphs[1])
pgraph.metadata['snt'] = 'This is a second running sentence.'

In [31]:
# Loop through all variables and select appropriate candidates for the new top variable
candidate_tops = pgraph.variables()
candidate_tops.remove( pgraph.top )

In [32]:
candidate_tops

{'o', 'r', 't'}

In [33]:
candidate_tops = [v for v in candidate_tops if incoming_edge_count(v, pgraph) == 0]

In [34]:
candidate_tops

['r']

In [35]:
pgraph.instances()

[Instance(source='s', role=':instance', target='sentence'),
 Instance(source='r', role=':instance', target='run-04'),
 Instance(source='o', role=':instance', target='ordinal-entity'),
 Instance(source='t', role=':instance', target='this')]

In [36]:
 # Create the list to try, keeping the original top first
new_tops = [pgraph.top] + candidate_tops
print(new_tops)
new_graphs = [penman.encode(pgraph, top=t) for t in new_tops]
print(new_graphs)

['s', 'r']
['# ::snt This is a second running sentence.\n(s / sentence\n   :ARG1-of (r / run-04\n               :ord (o / ordinal-entity\n                       :value 2))\n   :domain (t / this))', '# ::snt This is a second running sentence.\n(r / run-04\n   :ARG1 (s / sentence\n            :domain (t / this))\n   :ord (o / ordinal-entity\n           :value 2))']


In [37]:
# Get the mapping from top variables to the concept for debug
var2concept = {t.source:t.target for t in pgraph.instances()}
print(var2concept)

{'s': 'sentence', 'r': 'run-04', 'o': 'ordinal-entity', 't': 'this'}


In [38]:
gen_sents, _ = gtos.generate(new_graphs, disable_progress=True)
for sent, top in zip(gen_sents, new_tops):
    print('top: (%s / %s)' % (top, var2concept[top]))
    print('   ', sent)

top: (s / sentence)
    This is a second sentence running.
top: (r / run-04)
    This is the second sentence to run.


https://github.com/amrisi/amr-guidelines/blob/master/amr.md#amr-slogans ( regarding verbs and nouns)

https://github.com/amrisi/amr-guidelines/blob/master/amr.md#amr-slogans (modality)